检查给codebook引入Padding之后是否会影响模型性能

In [3]:
import random
import os
import transformers
import sys
sys.path.append("/workspace/zecheng/modelzipper/projects/custom_llama")
from dataclasses import dataclass, field
from transformers import Trainer
from modelzipper.tutils import *
from data.vqllama_dataset import VQDataCollator, VQLLaMAData
from models.vqvae import VQVAE
from data.svg_data import *
import pytorch_lightning as pl
from test import postprocess

VQVAE_CONFIG_PATH = "/workspace/zecheng/modelzipper/projects/custom_llama/configs/deepspeed/vqvae_config.yaml"
DATA_PATH = "/zecheng2/svg/icon-shop/test_data_snaps/test_mesh_data_svg_convert_p.pkl"

content = auto_read_data(DATA_PATH)
dataset = BasicDataset(dataset=content)
vqvae_config = load_yaml_config(VQVAE_CONFIG_PATH)

block_kwargs = dict(
        width=vqvae_config.vqvae_conv_block.width, 
        depth=vqvae_config.vqvae_conv_block.depth, 
        m_conv=vqvae_config.vqvae_conv_block.m_conv,
        dilation_growth_rate=vqvae_config.vqvae_conv_block.dilation_growth_rate,
        dilation_cycle=vqvae_config.vqvae_conv_block.dilation_cycle,
        reverse_decoder_dilation=vqvae_config.vqvae_conv_block.vqvae_reverse_decoder_dilation
    )

class PluginVQVAE(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

vqvae = VQVAE(vqvae_config, multipliers=None, **block_kwargs)
plugin_vqvae = PluginVQVAE(vqvae)
checkpoint = torch.load(vqvae_config.ckpt_path)  # load vqvae ckpt
plugin_vqvae.load_state_dict(checkpoint['state_dict'])
plugin_vqvae.eval()
plugin_vqvae.cpu()

begin to read data from /zecheng2/svg/icon-shop/test_data_snaps/test_mesh_data_svg_convert_p.pkl ...
begin to sanity check the dataset and conduct pre_process, num of samples: 2000, it will take some time...
load config files from /workspace/zecheng/modelzipper/projects/custom_llama/configs/deepspeed/vqvae_config.yaml
config loaded successfully!
config: namespace(ckpt_path='/zecheng2/vqllama/vqllama_quantizer/version_8/checkpoints/last.ckpt', pad_token_id=70, vqvae=namespace(levels=2, downs_t=[1, 1], strides_t=[2, 2], emb_width=4096, l_bins=8192, l_mu=0.99, spectral=0.0, multispectral=1.0, hvqvae_multipliers=[2, 1, 1], loss_fn='l2', dilation_growth_rate=1, use_nonrelative_specloss=True, use_bottleneck=True, commit=1.0, recon=1.0, linf_k=2048, use_modified_block=False), vqvae_conv_block=namespace(depth=4, width=512, m_conv=1.0, dilation_growth_rate=1, dilation_cycle=None, vqvae_reverse_decoder_dilation=True), dataset=namespace(max_path_nums=512, min_path_nums=4, pad_token_id=0, train_ba

<All keys matched successfully>

In [40]:
def cal_compress_padding_mask(x):

    # 确保长度是偶数，如果是奇数，可以添加一个值以配合压缩逻辑
    if len(x) % 2 != 0:
        x = torch.cat((x, torch.tensor([False])))

    # 压缩mask
    # 使用.view(-1, 2)将原始mask分为两列，然后使用.any(dim=1)检查每对是否有任何True值
    x = x.view(-1, 2).any(dim=1)
    
    return x

In [56]:
sample = dataset[0]['svg_path']
max_seq_len = 512
padded_sample = torch.concatenate([sample, torch.zeros(max_seq_len - sample.shape[0], 9)])
padding_mask = ~(padded_sample == 0).all(dim=1, keepdim=True).squeeze()
compress_padding_mask = cal_compress_padding_mask(padding_mask)
svg_token_ids, _ = plugin_vqvae.model.encode(padded_sample.unsqueeze(0), start_level=0, end_level=1)
svg_token_ids = svg_token_ids[0]  # 这里是不加padding mask的svg token ids

remain_svg_token_ids = svg_token_ids[:, :compress_padding_mask.sum()] # 这里是加入padding mask的svg token ids




ImportError: cannot import name 'postprocess' from 'test' (/opt/conda/envs/llama/lib/python3.10/test/__init__.py)

In [41]:
compress_padding_mask = cal_compress_padding_mask(padding_mask)

In [55]:
remain_svg_token_ids

tensor([[4769, 6022, 2332, 7758, 1569, 6788, 5531, 7700, 6174, 4948, 7938, 4768,
         7525, 4729, 7938, 7561, 3135, 4246, 4114, 8102, 3024, 5743, 5396, 5017,
         5853, 1010, 3793, 4587, 7054, 3964, 4413, 3545,  507, 2131, 6577, 3549,
         3431, 1318, 7670, 4729, 7149, 4534, 2645, 3456, 7525, 5071, 3070, 1404,
         3410, 3949, 5163, 2390, 4154, 6460, 7098, 7818, 2706, 5812, 7903, 5782,
          530, 2238,  291, 2219, 3717, 3410, 4328, 5163, 6670, 2634, 3785, 6468,
         6595, 2494, 5209]])

In [52]:
svg_token_ids[:compress_padding_mask.sum()]

tensor([[4769, 6022, 2332, 7758, 1569, 6788, 5531, 7700, 6174, 4948, 7938, 4768,
         7525, 4729, 7938, 7561, 3135, 4246, 4114, 8102, 3024, 5743, 5396, 5017,
         5853, 1010, 3793, 4587, 7054, 3964, 4413, 3545,  507, 2131, 6577, 3549,
         3431, 1318, 7670, 4729, 7149, 4534, 2645, 3456, 7525, 5071, 3070, 1404,
         3410, 3949, 5163, 2390, 4154, 6460, 7098, 7818, 2706, 5812, 7903, 5782,
          530, 2238,  291, 2219, 3717, 3410, 4328, 5163, 6670, 2634, 3785, 6468,
         6595, 2494, 5209, 5899, 7831, 1287, 6843,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  

In [45]:
svg_token_ids

tensor([[4769, 6022, 2332, 7758, 1569, 6788, 5531, 7700, 6174, 4948, 7938, 4768,
         7525, 4729, 7938, 7561, 3135, 4246, 4114, 8102, 3024, 5743, 5396, 5017,
         5853, 1010, 3793, 4587, 7054, 3964, 4413, 3545,  507, 2131, 6577, 3549,
         3431, 1318, 7670, 4729, 7149, 4534, 2645, 3456, 7525, 5071, 3070, 1404,
         3410, 3949, 5163, 2390, 4154, 6460, 7098, 7818, 2706, 5812, 7903, 5782,
          530, 2238,  291, 2219, 3717, 3410, 4328, 5163, 6670, 2634, 3785, 6468,
         6595, 2494, 5209, 5899, 7831, 1287, 6843,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,  728,
          728,  728,  728,  